In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris, load_wine, load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score

from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# Main Point
1) decision Tree & Bagging VS RandomForest  
2) 데이터셋별 성능 비교

In [20]:
# 데이터의 특성에 따른 앙상블 기법의 성능 차이를 위해 유방암, 아이리스꽃, 와인데이터셋 준비
cancer=load_breast_cancer()  #as_frame=True
# cancer=pd.DataFrame(cancerdata.data, columns=cancerdata.feature_names)
iris=load_iris()
wine=load_wine()

In [21]:
# print(cancer.isnull().sum())

In [22]:
# cancer.plot(kind='box', subplots=True, layout=(5, 6), figsize=(15, 21))
# plt.show()   

# Bagging을 사용한 분류
- bagging은 bootstrap aggregating의 줄임말
- 원래 훈련 데이터셋의 일부를 사용해 여러 모델을 훈련
- 각각의 결과를 결합해 최종 결과를 생성
- 분산을 줄이고 과적합을 막음
- 강력하고 복잡한 모델에서 잘 동작
- 주로 Decision Tree 에서 많이 사용하지만, 어떤 알고리즘에도 사용 가능하다
- 전체 데이터의 60~80% 밖에 활용하지 못함

## Decision Tree 

1) base model : DecisionTreeClassifier 사용한 모델  
2) bagging model : BaggingClassifier를 사용한 모델

In [44]:
base_model = make_pipeline(
StandardScaler(),
DecisionTreeClassifier(random_state=1)
)

- n_estimators = 모형의 갯수.= 한 예측기에 몇번의 시행을 할 것인지
- bootstrap = default=True. 데이터 중복 사용 허용여부
- max_samples= 하나의 예측기에 샘플이 랜덤하게 몇 %씩 들어가게 할 것인지 설정
- max_features=하나의 예측기가 가져갈 수 있는 최대의 칼럼 횟수 

▶ bagging을 적용한 Decision Tree가 Randomforest

In [45]:
bagging_model = BaggingClassifier(base_model, n_estimators=10, max_samples=0.5, max_features=0.5, random_state=1)

### base model VS bagging model 성능 비교

- fit_time : 훈련 시간
- score_time : 테스트 시간
- test_score : 훈련 점수
- train_score : 테스트 점수

- 데이터셋을 분리하는 방법  
  1) tain_test_split() 함수 사용  
   ex) X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, shuffle=True, random_state=1)

  2)   cross_validate() 함수 사용 
    - estimator: 평가하려는 모델
    - X : 훈련 데이터
    - y : 타깃 테이블
    - cv : 교차검증 분할 수 (k)

In [50]:
estimatorList=[base_model,bagging_model]

for i in estimatorList:
  cross_val = cross_validate(
  estimator=i,    
  X=cancer.data,
  y=cancer.target,
  cv=5,
  
  )
  # print('avg fit time: {:.5f}'.format(cross_val['fit_time'].mean()))
  # print('avg score time: {:.5f} '.format(cross_val['score_time'].mean()))
  print('avg test score: {:.5f} '.format(cross_val['test_score'].mean()))
  print(pd.DataFrame(cross_val))
  print()
  



avg test score: 0.91739 
   fit_time  score_time  test_score
0  0.010888    0.000688    0.903509
1  0.008966    0.000557    0.903509
2  0.008882    0.000540    0.921053
3  0.008721    0.000563    0.947368
4  0.009626    0.000560    0.911504

avg test score: 0.94547 
   fit_time  score_time  test_score
0  0.035705    0.002717    0.921053
1  0.034045    0.002666    0.956140
2  0.035434    0.002941    0.964912
3  0.035962    0.002653    0.964912
4  0.043608    0.005188    0.920354



## Random Forest

In [26]:
from sklearn.ensemble import RandomForestClassifier

In [51]:
base_model = make_pipeline(
StandardScaler(),
RandomForestClassifier(random_state=1)
)

In [54]:
cross_val = cross_validate(
estimator=base_model,    
X=cancer.data,
y=cancer.target,
cv=5,

)
# print('avg fit time: {:.5f}'.format(cross_val['fit_time'].mean()))
# print('avg score time: {:.5f} '.format(cross_val['score_time'].mean()))
print('avg test score: {:.5f} '.format(cross_val['test_score'].mean()))
print(pd.DataFrame(cross_val))

avg test score: 0.95959 
   fit_time  score_time  test_score
0  0.193749    0.012593    0.929825
1  0.211422    0.011088    0.947368
2  0.201972    0.010581    0.982456
3  0.195146    0.010492    0.973684
4  0.204167    0.010423    0.964602


# Boosting

### AdaBoost
- 대표적인 부스팅 알고리즘
- 일련의 약한 모델들을 학습
- 수정된 버전의 데이터를 반복 학습 (가중치가 적용된)
- 가중치 투표(또는 합)을 통해 각 모델의 예측 값을 결합
- 첫 단계에서는 원본 데이터를 학습하고 연속적인 반복마다 개별 샘플에 대한 
- 가중치가 수정되고 다시 모델이 학습
- 잘못 예측된 샘플은 가중치 증가, 올바르게 예측된 샘플은 가중치 감소
- 각각의 약한 모델들은 예측하기 어려운 샘플에 집중하게 됨


In [31]:
# 와인데이터는 성능이 비교적 떨어짐 -> 데이터 셋의 특성에 따라 성능이 달라짐

In [58]:
from sklearn.ensemble import AdaBoostClassifier

In [62]:
Adaboost_model = make_pipeline(
StandardScaler(),
AdaBoostClassifier()
)

In [60]:
cross_val = cross_validate(
estimator=Adaboost_model,    
X=cancer.data,
y=cancer.target,
cv=5,

)
# print('avg fit time: {:.5f}'.format(cross_val['fit_time'].mean()))
# print('avg score time: {:.5f} '.format(cross_val['score_time'].mean()))
print('avg test score: {:.5f} '.format(cross_val['test_score'].mean()))
print(pd.DataFrame(cross_val))

avg test score: 0.97187 
   fit_time  score_time  test_score
0  0.168521    0.009167    0.938596
1  0.157824    0.007499    0.973684
2  0.158443    0.009142    0.991228
3  0.158678    0.008194    0.991228
4  0.151001    0.007828    0.964602


# 성능 비교 한눈에 보기

In [67]:
datasets=[cancer, iris, wine]
estimators=[bagging_model,Adaboost_model ]

for data in datasets:
  print('--datasets--')
  for estimator in estimators:
    cross_val = cross_validate(
    estimator=estimator,    
    X=data.data,
    y=data.target,
    cv=5,

    )
    # print('avg fit time: {:.5f}'.format(cross_val['fit_time'].mean()))
    # print('avg score time: {:.5f} '.format(cross_val['score_time'].mean()))
    print('avg test score: {:.5f} '.format(cross_val['test_score'].mean()))
    print()

--datasets--
avg test score: 0.94547 

avg test score: 0.97187 

--datasets--
avg test score: 0.94000 

avg test score: 0.94667 

--datasets--
avg test score: 0.93286 

avg test score: 0.80857 



In [ ]:
var=y.predict-y.predict.mean()

y.predict-y

In [ ]:
# # 속성별 평균
# m = cancer.data.mean(axis=0)

# # 속성별 표준편차
# s = cancer.data.std(axis=0)

# # 정규확인 데이터
# data2 = (cancer.data - m)/s

# fig=plt.figure(figsize=[20,15])
# plt.boxplot(data2)
# pass

- 결과 해석
1. cancer, iris data의 경우 Adaboost_model의 성능이 더 좋음 -> Low variance, high bias
2. wine data의 경우 bagging_model의 성능이 더 좋음 -> High Variance, low bias

# XGBoost

- 트리 기반의 앙상블 기법
- 분류에 있어서 다른 알고리즘보다 좋은 예측 성능을 보여줌
- XGBoost는 GBM 기반이지만, GBM의 단점인 느린 수행 시간과 과적합 규제 부재 등의 문제를 해결
- 병렬 CPU 환경에서 빠르게 학습 가능
- kaggle 도전 데이터 과학자에서 5년 연속 1위한 알고리즘